In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import time
from datetime import datetime
import math


# ----------------------------
# 명시적 대기를 위한 라이브러리
# ----------------------------
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#### 명시적 대기

* `WebDriverWait(웹드라이버, 대기시간).until(조건함수)`
    * `visibility_of_element_located` : 요소가 DOM트리 상에 존재하고 화면에 표시되는지 여부
    * `element_to_be_clickable` : 요소가 클릭 가능한 상태인지 여부
        

#### 웹드라이버 생성 및 페이지 이동

In [2]:
driver = webdriver.Chrome()

# 웹페이지 접근
url = 'https://n.news.naver.com/article/005/0001707120'
driver.get(url)
driver.maximize_window()

#### 전체 댓글 수 추출

In [3]:
# ------------------------------------------------------------------------------------
# 전체 댓글 수 요소가 DOM트리에 존재하고 보여지고 클릭 가능할 때까지 대기하고 요소를 리턴받는다.
# ------------------------------------------------------------------------------------
element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#comment_count")))

# ------------------------------------------------------------------------------------
# 요소의 text가 '댓글'이 아닐 때까지 대기한다.
# ------------------------------------------------------------------------------------
WebDriverWait(driver, 10).until(lambda d: d.find_element(By.CSS_SELECTOR, "#comment_count").text != "댓글")

# ------------------------------------------------------------------------------------
# 요소의 text 추출
# ------------------------------------------------------------------------------------
review_total_cnt = element.text
print(review_total_cnt)

752


#### 댓글 화면으로 이동

In [4]:
# ------------------------------------------------------------------------------------
# 요소를 클릭하여 댓글 화면으로 이동한다.
# ------------------------------------------------------------------------------------
element.click()

#### 더보기 클릭하여 모든 댓글 보이기

In [5]:
# -----------------------------
# 더보기 클릭 횟수
#   더보기 클릭 할 때마다 20개의 댓글이 추가로 보여짐
#   ceil(전체 댓글 수/20)
# -----------------------------
pagecnt = math.ceil(int(review_total_cnt)/20)-1
# pagecnt = 3

In [6]:
for i in range(pagecnt):
    time.sleep(0.5)
    # ----------------------------------------------
    # 더보기 클릭 가능한 상태가 될때까지 대기
    # ----------------------------------------------
    print(f'{i}번째 페이지 로드중')
    element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".u_cbox_btn_more")))

    # ----------------------------------------------
    # 더보기 클릭
    # ----------------------------------------------
    element.click()
    
print('모든 댓글 로드 완료')

0번째 페이지 로드중
1번째 페이지 로드중
2번째 페이지 로드중
3번째 페이지 로드중
4번째 페이지 로드중
5번째 페이지 로드중
6번째 페이지 로드중
7번째 페이지 로드중
8번째 페이지 로드중
9번째 페이지 로드중
10번째 페이지 로드중
11번째 페이지 로드중
12번째 페이지 로드중
13번째 페이지 로드중
14번째 페이지 로드중
15번째 페이지 로드중
16번째 페이지 로드중
17번째 페이지 로드중
18번째 페이지 로드중
19번째 페이지 로드중
20번째 페이지 로드중
21번째 페이지 로드중
22번째 페이지 로드중
23번째 페이지 로드중
24번째 페이지 로드중
25번째 페이지 로드중
26번째 페이지 로드중
27번째 페이지 로드중
28번째 페이지 로드중
29번째 페이지 로드중
30번째 페이지 로드중
31번째 페이지 로드중
32번째 페이지 로드중
33번째 페이지 로드중
34번째 페이지 로드중
35번째 페이지 로드중
36번째 페이지 로드중
모든 댓글 로드 완료


#### 데이터 추출

In [7]:
# ------------------------------------------
# 댓글, 좋아요갯수, 싫어요갯수, 날짜
# ------------------------------------------

comments = driver.find_elements(By.CSS_SELECTOR, ".u_cbox_area")
comment_list = []
for comment in comments:
    try: review = comment.find_element(By.CSS_SELECTOR, ".u_cbox_contents").text
    except: review = np.nan

    try: reply_cnt = comment.find_element(By.CSS_SELECTOR, ".u_cbox_reply_cnt").text
    except: reply_cnt = np.nan

    try: recomm = comment.find_element(By.CSS_SELECTOR, ".u_cbox_cnt_recomm").text
    except: recomm = np.nan
    

    try: unrecomm = comment.find_element(By.CSS_SELECTOR, ".u_cbox_cnt_unrecomm").text
    except: unrecomm = np.nan

    try: date = comment.find_element(By.CSS_SELECTOR, ".u_cbox_date").text
    except: date = np.nan

    
    comment_list.append({"review":review, "reply_cnt":reply_cnt, 
                         "recomm":recomm, "unrecomm":unrecomm, "date":date})
                         

#### 데이터 저장

In [8]:
# -------------------------------
# 데이터프레임 생성하고 결측치 처리
# -------------------------------
df = pd.DataFrame(comment_list).dropna()

# -------------------------------
# 파일로 저장
# -------------------------------
file_path = f'data/뉴스댓글/나이키'
df.to_csv(file_path+'.csv', index=False)
df.to_excel(file_path+'.xlsx', index=False)

df

,review,reply_cnt,recomm,unrecomm,date
0,발볼이 넓은사람은 나이키가 불편해서 안신게됨,36,1095,34,2024.06.29. 10:15
1,난 나이키만 신는데? 신발은 나이키,47,968,381,2024.06.29. 09:59
3,디자인으로 신는거지 발이 편하진않음,15,321,37,2024.06.29. 10:02
4,발이 더 편하거나 디자인이 좋거나 하는 신발들이 많아졌지 사실 경쟁력이 없잖아,4,204,12,2024.06.29. 10:12
5,뉴발란스가 확실히 편해요. 이번에 아디다스 삼바도 사봤는데 나이키보다는 편하더라고요...,3,88,6,2024.06.29. 11:24
...,...,...,...,...,...
734,적지 않은 사람들이 짝퉁 신고 다니는 현실 감안하면 이해될만 합니다.,0,0,1,2024.06.29. 11:15
735,아직도 90년대에 사고가 멈춰있는 사람이 많구마잉,0,0,1,2024.06.29. 11:14
736,솔직히 그냥 줘도 안신음.,0,1,3,2024.06.29. 15:40
737,나이 키 묻는것은 굉장한 실례입니다,0,1,3,2024.06.29. 14:56
